In [27]:
import pandas as pd

In [28]:
trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset_2 August 2017\Trips\MattTestSurvey.csv')
rmove = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset_2 August 2017\Trips\MattTestRMove.csv')

#sample analysis, looks good
#trip.groupby('mode_1').count()
#trip.groupby('d_purp').count()
#trip.groupby('o_purp').count()

C:\Users\MJennings\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\MJennings\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,36,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
#Begin to make transformations

#In order of appearance in sample 2015 trip csv
final_trip = pd.DataFrame()
rmove_trip = pd.DataFrame()

#hhno
final_trip['hhno'] = trip['hhid']
rmove_trip['hhno'] = rmove['hhid']

#pno
final_trip['pno'] = trip['pernum']
rmove_trip['pno'] = rmove['pernum']

#day
final_trip['day'] = trip['daynum'].astype(int)
rmove_trip['day'] = rmove['daynum'].astype(int)

#tour
final_trip['tour'] = -1
rmove_trip['tour'] = -1

#half
final_trip['half'] = -1
rmove_trip['half'] = -1

#tseg
final_trip['tseg'] = -1
rmove_trip['tseg'] = -1

#tsvid
final_trip['tsvid'] = trip['tripnum']
rmove_trip['tsvid'] = rmove['tripnum']

#opurp
#using guidelines from Daysim2.1 Users Guide
def opurp (row):
   if row['o_purp'] == 1 :
      return 0
   if row['o_purp'] == 10 or row['o_purp'] == 11 or row['o_purp'] == 14 :
      return 1
   if row['o_purp'] == 6 :
      return 2
   if row['o_purp'] == 3 :
      return 3
   if row['o_purp'] == 33 or row['o_purp'] == 54 or row['o_purp'] == 56 or row['o_purp'] == 61 :
      return 4
   if row['o_purp'] == 30 or row['o_purp'] == 32 :
      return 5
   if row['o_purp'] == 50 :
      return 6
   if row['o_purp'] == 52 or row['o_purp'] == 62:
      return 7
   if row['o_purp'] == 53 or row['o_purp'] == 51 :
      return 8
   if row['o_purp'] == 34 :
      return 9
   if row['o_purp'] == 60 :
      return 10
   return 11

final_trip['opurp'] = trip.apply (lambda row: opurp (row),axis=1)
rmove_trip['opurp'] = rmove.apply (lambda row: opurp (row),axis=1)

#dpurp
def dpurp (row):
   if row['d_purp'] == 1 :
      return 0
   if row['d_purp'] == 10 or row['d_purp'] == 11 or row['d_purp'] == 14 :
      return 1
   if row['d_purp'] == 6 :
      return 2
   if row['d_purp'] == 3 :
      return 3
   if row['d_purp'] == 33 or row['d_purp'] == 54 or row['d_purp'] == 56 or row['d_purp'] == 61 :
      return 4
   if row['d_purp'] == 30 or row['d_purp'] == 32 :
      return 5
   if row['d_purp'] == 50 :
      return 6
   if row['d_purp'] == 52 or row['d_purp'] == 62:
      return 7
   if row['d_purp'] == 53 or row['d_purp'] == 51 :
      return 8
   if row['d_purp'] == 34 :
      return 9
   if row['d_purp'] == 60 :
      return 10
   return 11

final_trip['dpurp'] = trip.apply (lambda row: dpurp (row),axis=1)
rmove_trip['dpurp'] = rmove.apply (lambda row: dpurp (row),axis=1)

#oadtyp
final_trip['oadtyp'] = -1
rmove_trip['oadtyp'] = -1

#dadtyp
final_trip['dadtyp'] = -1
rmove_trip['dadtyp'] = -1

#opcl
final_trip['opcl'] = -1
rmove_trip['opcl'] = -1

#otaz
final_trip['otaz'] = -1
rmove_trip['otaz'] = -1

#dpcl
final_trip['dpcl'] = -1
rmove_trip['dpcl'] = -1

#dtaz
final_trip['dtaz'] = -1
rmove_trip['dtaz'] = -1

#mode (subject to revision, particularly with transit designations)
def mode (row):
   if row['mode_1'] == 1 :
      return 1
   if row['mode_1'] == 2 :
      return 2
   if (row['mode_1'] in range(3,23) or row['mode_1'] in range(33,38)) and row['travelers_total'] == 1:
      return 3
   if (row['mode_1'] in range(3,23) or row['mode_1'] in range(33,38)) and row['travelers_total'] == 2:
      return 4
   if (row['mode_1'] in range(3,23) or row['mode_1'] in range(33,38)) and row['travelers_total'] > 2:
      return 5
   if row['mode_1'] == 23 or row['mode_1'] == 32 or row['mode_1'] == 52 or row['mode_1'] in range(26,29) or row['mode_1'] in range(41,43):
      return 6
   if row['mode_1'] == 24 :
      return 8
   return 9

final_trip['mode'] = trip.apply (lambda row: mode (row),axis=1)
rmove_trip['mode'] = rmove.apply (lambda row: mode (row),axis=1)

#pathtype (removing network options: 1 = local bus, 2 = light rail, 3 = premium bus, 4 = commuter rail, 5 = ferry/water taxi)
def pathtype (row):
   if row['mode_1'] == 23 :
      return 1
   if row['mode_1'] == 52 :
      return 2
   if row['mode_1'] == row['mode_1'] in range(26,29) :
      return 3
   if row['mode_1'] == row['mode_1'] in range(41,43) :
      return 4 
   if row['mode_1'] == 32 :
      return 5  
   return 0

final_trip['pathtype'] = trip.apply (lambda row: pathtype (row),axis=1)
rmove_trip['pathtype'] = rmove.apply (lambda row: pathtype (row),axis=1)

#dorp
def dorp (row):
   if row['driver'] == 1 :
      return 1
   if row['driver'] == 2 :
      return 2
   return 3
    
final_trip['dorp'] = trip.apply (lambda row: dorp (row),axis=1)
rmove_trip['dorp'] = rmove.apply (lambda row: dorp (row),axis=1)

#deptm
final_trip['deptm'] = trip['depart_time_mam']
rmove_trip['deptm'] = rmove['depart_time_mam']

#arrtm
final_trip['arrtm'] = trip['arrival_time_mam']
rmove_trip['arrtm'] = rmove['arrival_time_mam']

#endacttm
trip['a_dur'].fillna(0, inplace=True)
final_trip['endacttm'] = trip['a_dur'].astype(int)
rmove['a_dur'].fillna(0, inplace=True)
rmove_trip['endacttm'] = rmove['a_dur'].astype(int)

#travtime (best guess)
trip['google_duration'].fillna(0, inplace=True)
final_trip['travtime'] = trip['google_duration'].astype(int)
rmove['google_duration'].fillna(0, inplace=True)
rmove_trip['travtime'] = rmove['google_duration'].astype(int)

#travcost (best guess)
def toll (row):
   if row['toll_pay'] == 3 :
      return 1
   if row['toll_pay'] == 4 :
      return 3
   if row['toll_pay'] == 5 :
      return 5 
   if row['toll_pay'] == 6 :
      return 7  
   if row['toll_pay'] == 7 :
      return 9   
   if row['toll_pay'] == 8 :
      return 10    
   return 0

final_trip['travcost'] = trip.apply (lambda row: toll (row),axis=1)
rmove_trip['travcost'] = rmove.apply (lambda row: toll (row),axis=1)

#travdist (best guess)
final_trip['travdist'] = trip['trip_path_distance']
rmove_trip['travdist'] = rmove['trip_path_distance']



In [30]:
final_trip
rmove_trip

,hhno,pno,day,tour,half,tseg,tsvid,opurp,dpurp,oadtyp,...,dtaz,mode,pathtype,dorp,deptm,arrtm,endacttm,travtime,travcost,travdist
0,17100059,1,1,-1,-1,-1,1,0,1,-1,...,-1,6,3,3,530,573,456,44,0,8.69
1,17100059,1,1,-1,-1,-1,2,1,10,-1,...,-1,6,3,3,1029,1039,0,10,0,0.65
2,17100059,1,1,-1,-1,-1,3,10,10,-1,...,-1,6,3,3,1039,1048,0,9,0,0.84
3,17100059,1,1,-1,-1,-1,4,10,10,-1,...,-1,6,3,3,1048,1089,0,41,0,13.70
4,17100059,1,1,-1,-1,-1,5,10,0,-1,...,-1,6,3,3,1089,1094,836,5,0,0.51
5,17100059,1,2,-1,-1,-1,6,0,9,-1,...,-1,6,1,3,1930,1970,60,40,0,6.41
6,17100059,1,2,-1,-1,-1,7,9,1,-1,...,-1,6,1,3,2030,2080,510,50,0,2.51
7,17100059,1,2,-1,-1,-1,8,1,8,-1,...,-1,3,0,2,2590,2610,187,20,0,10.40
8,17100059,1,2,-1,-1,-1,9,8,10,-1,...,-1,1,0,3,2797,2811,4,14,0,0.73
9,17100059,1,2,-1,-1,-1,10,10,0,-1,...,-1,3,0,3,2815,2834,578,19,0,14.48


In [31]:
final_trip.to_csv('testresults.csv')